In [30]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.0.0


In [31]:
from tensorflow.python.client import device_lib

def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

# 选择编号为1的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']


In [32]:
# 获取gpus
tf.test.is_gpu_available()
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_gpu_available()

Num GPUs Available:  4


True

In [33]:
train_dir = '../../datasets/10-monkey/training'
valid_dir = '../../datasets/10-monkey/validation'
label_file = '../../datasets/10-monkey/monkey_labels.txt'

print(os.path.exists(train_dir))

True


In [34]:
labels = pd.read_csv(label_file,header=0)
labels

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26
5,n5,cebus_capucinus\t,white_headed_capuchin,141,28
6,n6,mico_argentatus\t,silvery_marmoset,132,26
7,n7,saimiri_sciureus\t,common_squirrel_monkey,142,28
8,n8,aotus_nigriceps\t,black_headed_night_monkey,133,27
9,n9,trachypithecus_johnii,nilgiri_langur,132,26


In [35]:
height = 128 # 图片缩放到的大小
width =128
channels = 3 
batch_size = 64
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255.,  # 归一化
    rotation_range=40, #随机旋转40度之内
    width_shift_range=0.2,# 水平位移
    height_shift_range=0.2, # 竖直位移 比例
    shear_range=0.2,
    zoom_range=0.2,#缩放程度
    horizontal_flip=True,#水平翻转
    fill_mode='nearest', # 填充像素规则
)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(height,width),
                                                  batch_size=batch_size,
                                                  seed=7,
                                                  shuffle=True,
                                                  class_mode='categorical')

valid_datagen =keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                   target_size=(height,width),
                                                  batch_size=batch_size,
                                                  seed=7,
                                                  class_mode='categorical')

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num,valid_num)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098 272


In [36]:
for i in range(2):
    x,y = train_generator.next()
    print(x.shape,y.shape)
    print(y)

(64, 128, 128, 3) (64, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.

In [37]:
model  = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',
                       input_shape = [width,height,channels]),
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='selu'),
    keras.layers.Dense(10,activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 128)      

In [38]:
epochs =10
history = model.fit_generator(train_generator,steps_per_epoch=train_num//batch_size,
                             epochs=epochs,validation_data=valid_generator,
                             validation_steps=valid_num//batch_size)

Epoch 1/10
17/17 [==============================] - 31s 2s/step - loss: 2.3821 - accuracy: 0.1054 - val_loss: 2.2627 - val_accuracy: 0.1367
Epoch 2/10
 7/17 [===========>..................] - ETA: 12s - loss: 2.2682 - accuracy: 0.1607

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())

def plot_learning_curves(history,label,epochs,min_value,max_value):
    data = {}
    data[label]=history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8,5))
    plt.grid(True)
    plt.axis([0,epochs,min_value,max_value])
    plt.show()
    
plot_learning_curves(history,'accuracy',epochs,0,1)
plot_learning_curves(history,'loss',epochs,1.2,2.5)